<a href="https://colab.research.google.com/github/siddharthaborgohain/Tutorials-Python-In-Hindi/blob/main/Pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df=pd.read_csv("http://bit.ly/kaggletrain")

In [3]:
df.shape

(891, 12)

In [4]:
df.columns

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')

In [5]:
df.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [6]:
df=df.loc[df.Embarked.notna(),['Survived','Pclass','Sex','Embarked']]

In [7]:
df.isna().sum()

Survived    0
Pclass      0
Sex         0
Embarked    0
dtype: int64

In [8]:
df.head()

,Survived,Pclass,Sex,Embarked
0,0,3,male,S
1,1,1,female,C
2,1,3,female,S
3,1,1,female,S
4,0,3,male,S


In [9]:
X=df.loc[:,['Pclass']]
y=df.Survived

In [10]:
X.shape

(889, 1)

In [11]:
y.shape

(889,)

In [12]:
from sklearn.linear_model import LogisticRegression

In [13]:
logreg=LogisticRegression(solver='lbfgs')

In [14]:
from sklearn.model_selection import cross_val_score

In [15]:
cross_val_score(logreg, X, y, cv=5, scoring='accuracy').mean()

0.6783406335301212

In [18]:
y.value_counts(normalize=True)

0    0.617548
1    0.382452
Name: Survived, dtype: float64

In [19]:
df.head()

,Survived,Pclass,Sex,Embarked
0,0,3,male,S
1,1,1,female,C
2,1,3,female,S
3,1,1,female,S
4,0,3,male,S


In [21]:
from sklearn.preprocessing import OneHotEncoder
ohe=OneHotEncoder(sparse=False)

In [26]:
ohe.fit_transform(df[['Embarked']])

array([[0., 0., 1.],
       [1., 0., 0.],
       [0., 0., 1.],
       ...,
       [0., 0., 1.],
       [1., 0., 0.],
       [0., 1., 0.]])

In [27]:
ohe.categories_

[array(['C', 'Q', 'S'], dtype=object)]

In [25]:
## Beter Technique

In [28]:
X = df.drop('Survived',axis='columns')

In [29]:
X.head()

,Pclass,Sex,Embarked
0,3,male,S
1,1,female,C
2,3,female,S
3,1,female,S
4,3,male,S


In [31]:
from sklearn.compose import make_column_transformer

In [32]:
ct= make_column_transformer(
    (OneHotEncoder(),['Sex','Embarked'])
    ,remainder='passthrough')

In [33]:
ct.fit_transform(X)

array([[0., 1., 0., 0., 1., 3.],
       [1., 0., 1., 0., 0., 1.],
       [1., 0., 0., 0., 1., 3.],
       ...,
       [1., 0., 0., 0., 1., 3.],
       [0., 1., 1., 0., 0., 1.],
       [0., 1., 0., 1., 0., 3.]])

In [34]:
from sklearn.pipeline import make_pipeline

In [35]:
pipe=make_pipeline(ct,logreg)

In [36]:
cross_val_score(pipe, X, y, cv=5, scoring='accuracy').mean()

0.7727924839713071

In [37]:
X_new= X.sample(5, random_state=99)
X_new

,Pclass,Sex,Embarked
599,1,male,C
512,1,male,S
273,1,male,C
215,1,female,C
790,3,male,Q


In [38]:
pipe.fit(X,y)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('onehotencoder',
                                                  OneHotEncoder(),
                                                  ['Sex', 'Embarked'])])),
                ('logisticregression', LogisticRegression())])

In [39]:
pipe.predict(X_new)

array([1, 0, 1, 1, 0])